# Assignment Analyser
using tensorflow and opencv 

(@kamaravichow)

## Preparing data

As official mnist datasets doesn't have operations like "+", "=", we need to generate our own dataset. For that we'll use the fonts on our computer that is used to make the questions of the assignment


In [1]:
# coding=utf-8

In [2]:
from __future__ import print_function
from PIL import Image, ImageFont, ImageDraw
import os
import shutil
import time

In [3]:
lable_dict = {0: '0', 1:'1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8:'8', 9:'9', 10:'=', 11:'+', 12:'-', 13:'×', 14: '÷'}

In [4]:
for value,char in lable_dict.items():
    print(value,char)
    train_images_dir = './train_images/' + str(value)
    if os.path.isdir(train_images_dir):
        shutil.rmtree(train_images_dir)
    os.makedirs(train_images_dir)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 =
11 +
12 -
13 ×
14 ÷


In [5]:
def generate_image(lable_dict, font_path, w=24, h=24, rotation =0):

    for value,char in lable_dict.items():
        img = Image.new('RGB', (w, h), "black")
        draw = ImageDraw.Draw(img)

        font = ImageFont.truetype(font_path, int(w * 0.9))
        font_width, font_height = draw.textsize(char, font)

        x = (w - font_width - font.getoffset(char)[0]) / 2
        y = (h - font_height - font.getoffset(char)[1]) / 2

        draw.text((x, y), char, (255,255,255), font)

        img = img.rotate(rotation)

        time_value = int(round(time.time() * 1000))
        img_path = "./train_images/{}/img-{}_r-{}_{}.png".format(value, value, rotation, time_value)
        img.save(img_path)


In [8]:
font_dir = 'F:/Desktop/homework-analyser-python/generator/fonts'

for font_name in os.listdir(font_dir):
    path_of_font = os.path.join(font_dir, font_name)
    
    for k in range(-10,10,1):
        generate_image(lable_dict, path_of_font, rotation=k)

The above will generate the dataset from font files to ./train_images

## Building the model

In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import cv2

In [10]:
def create_model():
    model = Sequential([
        layers.experimental.preprocessing.Rescaling(1./255, input_shape=(24,24,1)),
        layers.Conv2D(24,3,activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,3, activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(15)
    ])

    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

    return model


model = create_model()

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 24, 24, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 22, 22, 24)        240       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          13888     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

### Training the model on dataset

In [12]:
dataset_dir = pathlib.Path('F:/Desktop/homework-analyser-python/generator/train_images')
print(dataset_dir.name)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    color_mode="grayscale",
    image_size=(24,24),
    batch_size=32
)

train_images
Found 600 files belonging to 15 classes.


In [13]:
class_names = train_ds.class_names

np.save("class_name.npy", class_names)

In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

model = create_model()

model.fit(train_ds, epochs=10)

model.save_weights('checkpoint/char_checkpoint')

Epoch 1/10
19/19 [==============================] - 1s 13ms/step - loss: 2.2733 - accuracy: 0.4017
Epoch 2/10
19/19 [==============================] - 0s 14ms/step - loss: 0.8380 - accuracy: 0.8917
Epoch 3/10
19/19 [==============================] - 0s 14ms/step - loss: 0.1610 - accuracy: 0.9717
Epoch 4/10
19/19 [==============================] - 0s 12ms/step - loss: 0.0498 - accuracy: 0.9950
Epoch 5/10
19/19 [==============================] - 0s 10ms/step - loss: 0.0193 - accuracy: 1.0000
Epoch 6/10
19/19 [==============================] - 0s 10ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 7/10
19/19 [==============================] - 0s 9ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 8/10
19/19 [==============================] - ETA: 0s - loss: 0.0045 - accuracy: 1.00 - 0s 11ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 9/10
19/19 [==============================] - 0s 13ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 10/10
19/19 [==============================] - 0s 11ms/step - 

### Prediction from the model

In [18]:
img1 = cv2.imread('img1.png', 0)
img2 = cv2.imread('img2.png', 0)

In [19]:
imgs = np.array([img1, img2])

In [20]:
imgs

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)

In [21]:
model = create_model()
model.load_weights('F:/Desktop/homework-analyser-python/model/checkpoint/char_checkpoint')
class_name = np.load('class_name.npy')

In [22]:
predicted = model.predict(imgs)

results = []
for predict in predicted:
    index = np.argmax(predict)
    result = class_name[index]
    results.append(result)

results

['8', '14']